## Khởi tạo môi trường

In [1]:
import os 

os.environ["PYSPARK_PYTHON"]="python3.7"
os.environ["PYSPARK_DRIVER_PYTHON"]="python3.7"

## Tạo phiên spark

In [2]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml import PipelineModel
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)
# spark = SparkSession \
#         .builder \
#         .master("spark://spark-master:7077") \
#         .appName("FP-Growth-coppy") \
#         .getOrCreate()

## Đọc dữ liệu từ HDFS

In [3]:
df = spark.read.format('csv').option("inferSchema", "true").load("hdfs://namenode/user/taipt/clustered")

In [4]:
df_date = spark.read.format('csv').option("inferSchema", "true").load("hdfs://namenode/user/taipt/customerDate")

In [5]:
df_date.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: timestamp (nullable = true)



In [6]:
df_date=df_date.withColumn("InvoiceDate", col("_c1"))
df_date=df_date.select(col('_c0'), col('InvoiceDate'))
df_date=df_date.filter(col("InvoiceDate").between('2011-06-09 00:00:00','2011-12-09 00:00:00 '))
df_date=df_date.na.drop().dropDuplicates()

In [7]:
df_date=df_date.select(col('_c0'), col('InvoiceDate'))

In [61]:
df_date=df_date.filter(col("InvoiceDate").between('2011-06-09 00:00:00','2011-12-09 00:00:00 '))

In [62]:
df_date=df_date.na.drop().dropDuplicates()

In [63]:
# df_date.show(3)

+-----+-------------------+
|  _c0|        InvoiceDate|
+-----+-------------------+
|15021|2011-06-17 12:16:00|
|17389|2011-08-03 10:12:00|
|17585|2011-06-22 13:28:00|
+-----+-------------------+
only showing top 3 rows



In [13]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)



In [14]:
df.withColumn("CustomerID", col("_c0")).withColumn("InvoiceNo", col("_c1")).withColumn("Description", col("_c2")).withColumn("Cluster", col("_c3"))

DataFrame[_c0: int, _c1: int, _c2: string, _c3: int, CustomerID: int, InvoiceNo: int, Description: string, Cluster: int]

In [15]:
df_after=df

In [16]:
# df.show()

## Tiền xử lý dữ liệu

In [17]:
df = df.groupBy("_c0", "_c1", "_c3").agg(collect_set("_c2").alias("items"))

In [18]:
df = df.filter((df._c3=="0"))

In [19]:
df.printSchema()


root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [20]:
df.show()

+-----+------+---+--------------------+
|  _c0|   _c1|_c3|               items|
+-----+------+---+--------------------+
|12951|553909|  0|[JUMBO BAG DOILEY...|
|14037|557038|  0|[FELTCRAFT DOLL E...|
|14483|545470|  0|[MONEY BOX BISCUI...|
|14708|537350|  0|[MULTI COLOUR SIL...|
|15063|552644|  0|[REGENCY TEAPOT R...|
|15555|574673|  0|[WOODLAND MINI BA...|
|15572|548952|  0|[EASTER TIN CHICK...|
|16712|579686|  0|[IVORY CAFE HANGI...|
|17315|564976|  0|[72 SWEETHEART FA...|
|17720|576056|  0|[15CM CHRISTMAS G...|
|17757|549572|  0|[BLUE TEA TOWEL C...|
|17932|575729|  0|[SET OF 6 T-LIGHT...|
|18118|555631|  0|[RIBBON REEL HEAR...|
|14071|580876|  0|[SMALL YELLOW BAB...|
|14409|537410|  0|[HAND WARMER BABU...|
|14560|545989|  0|[SWEETHEART CAKES...|
|15150|574723|  0|[SET OF 6 RIBBONS...|
|15812|542212|  0|[EDWARDIAN PARASO...|
|16880|555508|  0|[FANCY FONT BIRTH...|
|17135|571641|  0|[SWISS CHALET TRE...|
+-----+------+---+--------------------+
only showing top 20 rows



In [21]:
list_item_of_user=df.select("items").where(df._c0==12951).collect()

In [22]:
list_item_of_user

[Row(items=['JUMBO BAG DOILEY PATTERNS', 'SPOTTY BUNTING', 'PAPER BUNTING RETROSPOT', 'PAPER BUNTING VINTAGE PAISLEY', 'VINTAGE UNION JACK BUNTING', 'SET OF 10 LED DOLLY LIGHTS', 'RECYCLING BAG RETROSPOT', 'PAPER CHAIN KIT VINTAGE CHRISTMAS', 'WOODEN REGATTA BUNTING']),
 Row(items=['WOODEN UNION JACK BUNTING', 'STARS GIFT TAPE', 'TEA TIME PARTY BUNTING', 'VINTAGE UNION JACK BUNTING', 'PAPER BUNTING RETROSPOT', 'GROW YOUR OWN BASIL IN ENAMEL MUG']),
 Row(items=['LARGE CAKE TOWEL PINK SPOTS', 'WASHROOM METAL SIGN', 'SMALL HEART MEASURING SPOONS', 'CALENDAR PAPER CUT DESIGN', 'T-LIGHT HOLDER SILVER HEART HANDLE', 'LAUNDRY 15C METAL SIGN', 'JAM MAKING SET WITH JARS']),
 Row(items=['FELTCRAFT 6 FLOWER FRIENDS', 'PINK HEART SHAPE EGG FRYING PAN', '6 RIBBONS EMPIRE', 'ZINC WILLIE WINKIE  CANDLE STICK', 'WOODEN HAPPY BIRTHDAY GARLAND', 'HAPPY EASTER HANGING DECORATION', 'HOMEMADE JAM SCENTED CANDLES', 'ELEPHANT, BIRTHDAY CARD,', 'PINK BLUE FELT CRAFT TRINKET BOX', '10 COLOUR SPACEBOY PEN', 'SI

## FP-Growth

In [23]:
from pyspark.ml.fpm import FPGrowth

In [24]:
df = df.select("_c0", "_c1", "items")

In [25]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.02)

In [26]:
model = fpGrowth.fit(df)

### Display frequent itemsets

In [27]:
model.freqItemsets.show(truncate=False)

+------------------------------------------------------------------------------------+----+
|items                                                                               |freq|
+------------------------------------------------------------------------------------+----+
|[CANDLEHOLDER PINK HANGING HEART]                                                   |270 |
|[CANDLEHOLDER PINK HANGING HEART, WHITE HANGING HEART T-LIGHT HOLDER]               |203 |
|[PINK FLORAL FELTCRAFT SHOULDER BAG]                                                |184 |
|[PINK BABY BUNTING]                                                                 |137 |
|[WHITE HANGING HEART T-LIGHT HOLDER]                                                |1567|
|[ASSORTED COLOUR BIRD ORNAMENT]                                                     |1109|
|[ASSORTED COLOUR BIRD ORNAMENT, WHITE HANGING HEART T-LIGHT HOLDER]                 |221 |
|[PLASTERS IN TIN CIRCUS PARADE]                                                

### Display generated association rules

In [28]:
model.associationRules.show(truncate=False)

+------------------------------------------------+-------------------------------------+-------------------+------------------+
|antecedent                                      |consequent                           |confidence         |lift              |
+------------------------------------------------+-------------------------------------+-------------------+------------------+
|[HOT WATER BOTTLE KEEP CALM]                    |[SCOTTIE DOG HOT WATER BOTTLE]       |0.2807625649913345 |6.700933880278792 |
|[HOT WATER BOTTLE KEEP CALM]                    |[CHOCOLATE HOT WATER BOTTLE]         |0.32062391681109187|7.967564374687621 |
|[HOT WATER BOTTLE KEEP CALM]                    |[HOT WATER BOTTLE I AM SO POORLY]    |0.2616984402079723 |8.511613484215177 |
|[HOT WATER BOTTLE KEEP CALM]                    |[HOT WATER BOTTLE TEA AND SYMPATHY]  |0.30329289428076256|10.240958542253738|
|[HOT WATER BOTTLE KEEP CALM]                    |[LOVE HOT WATER BOTTLE]              |0.28942807625649

In [29]:
df_result = model.associationRules

In [30]:
df_result.show()

+--------------------+--------------------+-------------------+------------------+
|          antecedent|          consequent|         confidence|              lift|
+--------------------+--------------------+-------------------+------------------+
|[HOT WATER BOTTLE...|[SCOTTIE DOG HOT ...| 0.2807625649913345| 6.700933880278792|
|[HOT WATER BOTTLE...|[CHOCOLATE HOT WA...|0.32062391681109187| 7.967564374687621|
|[HOT WATER BOTTLE...|[HOT WATER BOTTLE...| 0.2616984402079723| 8.511613484215177|
|[HOT WATER BOTTLE...|[HOT WATER BOTTLE...|0.30329289428076256|10.240958542253738|
|[HOT WATER BOTTLE...|[LOVE HOT WATER B...|0.28942807625649913|10.788512842482424|
|[IVORY DINER WALL...|[RED DINER WALL C...|  0.493006993006993|22.715981934731936|
|[LUNCH BAG SUKI D...|[LUNCH BAG SPACEB...| 0.5194029850746269|  9.87460975922679|
|[LUNCH BAG SUKI D...|[LUNCH BAG CARS B...| 0.5194029850746269| 9.818344176553133|
|[LUNCH BAG SUKI D...|[LUNCH BAG RED RE...|                0.6|  9.45605700712589|
|[LU

In [31]:
df_result.createOrReplaceTempView("dfTable")


final_result=spark.sql( "select  consequent from dfTable " )

In [32]:
final_result.show()

+--------------------+
|          consequent|
+--------------------+
|[SCOTTIE DOG HOT ...|
|[CHOCOLATE HOT WA...|
|[HOT WATER BOTTLE...|
|[HOT WATER BOTTLE...|
|[LOVE HOT WATER B...|
|[RED DINER WALL C...|
|[LUNCH BAG SPACEB...|
|[LUNCH BAG CARS B...|
|[LUNCH BAG RED RE...|
|[LUNCH BAG PINK P...|
|[LUNCH BAG APPLE ...|
|[LUNCH BAG WOODLAND]|
|[PAPER CHAIN KIT ...|
|[SET OF 20 VINTAG...|
|[60 CAKE CASES VI...|
|[JUMBO BAG PINK P...|
|[JUMBO BAG RED RE...|
|[JUMBO BAG VINTAG...|
|[JUMBO STORAGE BA...|
|[JUMBO  BAG BAROQ...|
+--------------------+
only showing top 20 rows



### Transform examines the input items against all the association rules and summarize the consequents as prediction


In [33]:
df_predic=model.transform(df)

In [34]:
# df_predic.show()

In [35]:
rfm_table = df_predic.join(df_date, ['_c0'])

In [36]:
rfm_table = rfm_table.na.drop().dropDuplicates()

In [37]:
rfm_table.createOrReplaceTempView("dfTableResult")

In [38]:
df_x=spark.sql('SELECT  max(InvoiceDate) as date from dfTableResult ')


In [39]:
rfm_table.show()

+-----+------+--------------------+--------------------+-------------------+
|  _c0|   _c1|               items|          prediction|        InvoiceDate|
+-----+------+--------------------+--------------------+-------------------+
|12940|571270|[LAUREL HEART ANT...|[WOODEN PICTURE F...|2011-10-16 12:09:00|
|12940|571270|[LAUREL HEART ANT...|[WOODEN PICTURE F...|2011-09-13 10:16:00|
|12940|566488|[SET OF 3 REGENCY...|[RED DINER WALL C...|2011-10-16 12:09:00|
|12940|566488|[SET OF 3 REGENCY...|[RED DINER WALL C...|2011-09-13 10:16:00|
|13285|576771|[60 CAKE CASES VI...|[LUNCH BAG SPACEB...|2011-11-16 13:19:00|
|13285|576771|[60 CAKE CASES VI...|[LUNCH BAG SPACEB...|2011-07-01 14:53:00|
|13285|576771|[60 CAKE CASES VI...|[LUNCH BAG SPACEB...|2011-04-27 13:03:00|
|13285|576771|[60 CAKE CASES VI...|[LUNCH BAG SPACEB...|2011-02-22 17:13:00|
|13285|544685|[HANGING JAM JAR ...|[LUNCH BAG  BLACK...|2011-11-16 13:19:00|
|13285|544685|[HANGING JAM JAR ...|[LUNCH BAG  BLACK...|2011-07-01 14:53:00|

In [40]:
df_result.createOrReplaceTempView("dfTable")
df_re=spark.sql("select * from dfTable")


In [41]:
df_re.show()

+--------------------+--------------------+-------------------+------------------+
|          antecedent|          consequent|         confidence|              lift|
+--------------------+--------------------+-------------------+------------------+
|[HOT WATER BOTTLE...|[SCOTTIE DOG HOT ...| 0.2807625649913345| 6.700933880278792|
|[HOT WATER BOTTLE...|[CHOCOLATE HOT WA...|0.32062391681109187| 7.967564374687621|
|[HOT WATER BOTTLE...|[HOT WATER BOTTLE...| 0.2616984402079723| 8.511613484215177|
|[HOT WATER BOTTLE...|[HOT WATER BOTTLE...|0.30329289428076256|10.240958542253738|
|[HOT WATER BOTTLE...|[LOVE HOT WATER B...|0.28942807625649913|10.788512842482424|
|[IVORY DINER WALL...|[RED DINER WALL C...|  0.493006993006993|22.715981934731936|
|[LUNCH BAG SUKI D...|[LUNCH BAG SPACEB...| 0.5194029850746269|  9.87460975922679|
|[LUNCH BAG SUKI D...|[LUNCH BAG CARS B...| 0.5194029850746269| 9.818344176553133|
|[LUNCH BAG SUKI D...|[LUNCH BAG RED RE...|                0.6|  9.45605700712589|
|[LU

In [42]:
df_convertArray=np.array(df_re)

NameError: name 'np' is not defined

In [ ]:
df_after.createOrReplaceTempView("dfTableFirst")

In [ ]:
df_1=spark.sql("select  max(lift) from dfTable where antecedent in ( select _c2 from dfTableFirst where _c0='12951' ) group by antecedent  " )
df_1.show()